# Capstone Project: BRICS Analysis Data Preparation
### By Ryan Kidd

## Introduction
For my Capstone Project, I want to analyze how the different aspects of the BRICS (Brazil, Russia, India, China, and South Africa) economies distinguished these countries as the strongest emerging markets in the twentieth century. These economies have the potential to become the world's fastest economies with their low labor costs, locational advantages, and abundant natural resources, yet why haven't they? Did political instability cause economic struggle in these countries, or are there underlying reasons in the data that suggest a particular industry underperformed? Using data science techniques, I will analyze the different industries that contribute the most to the BRICS economy overtime. Further, I want to analyze how different aspects of the BRICS economy contributed to their overall GDP and Growth per Year. Through machine learning techniques, I want to answer why some of these countries underperformed, or the underlying cause as to why they were not able to meet expectations.

## Data Preparation
In order to analyze the BRICS datasets, first I will need to prepare the data and then clean it appropriately. I will look into the five datasets downloaded from Kaggle, `https://www.kaggle.com/docstein/brics-world-bank-indicators` and understand what the columns are representing. For each dataset, I will analyze what the datatypes of the columns are and check for number of null values. Depending on how many null values and the proportion of data they account for, I will decide whether to drop them or impute them. Further, I plan on merging the datasets and rotating them in order to achieve a cumulative dataset that will allow me to conduct different machine learning models. This rotated dataset will have the rows as the BRICS countries and the features as the different Series in each dataset. I will also need to create some features that will help me in my analysis such as a Increase/Decrease per Year, Percent Growth, and Total GDP. First, in order to construct the data in this way, I must read in these files. Below, I have downloaded the necessary packages to prepare and clean the datasets.

In [24]:
# Import data science packages for Data Preparation, Cleaning, and Exploratory Data Analysis (EDA)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [25]:
# Reading in all of the CSV files Kaggle for BRICS analysis
df_economy = pd.read_csv('data/Economy_Data.csv', sep=';', index_col=0) # Had to add sep = ";" as all of the CSV file columns were separated with a ";" 
df_education = pd.read_csv('data/Education_And_Environ_Data.csv', sep=';', index_col=0) # Got rid of the index_column as it provides no useful information
df_public = pd.read_csv('data/Public_Sector_Indicators.csv', sep=';', index_col=0)
df_private = pd.read_csv('data/Private_Sector_Data.csv', sep=';', index_col=0)
df_health = pd.read_csv('data/Health_And_Poverty_Data.csv', sep=';', index_col=0)

In [26]:
# Created new Excel files with columns separated by exporting a pandas Dataframe to CSV
# df_economy.to_csv(r'C:\Users\Jim\Desktop\Economy_Fixed.csv', index = False)
# df_education.to_csv(r'C:\Users\Jim\Desktop\Education_Fixed.csv', index = False)
# df_public.to_csv(r'C:\Users\Jim\Desktop\Public_Fixed.csv', index = False)
# df_private.to_csv(r'C:\Users\Jim\Desktop\Private_Fixed.csv', index = False)
# df_health.to_csv(r'C:\Users\Jim\Desktop\Health_Fixed.csv', index = False)

In [27]:
# Taking a look into the Economy_Data.csv
df_economy.head(10) #Looking at the first 10 rows

,SeriesCode,CountryName,CountryCode,Year,Value
SeriesName,,,,,
Adjusted net national income (annual % growth),NY.ADJ.NNTY.KD.ZG,Brazil,BRA,1970.0,NaN
Adjusted net national income (annual % growth),NY.ADJ.NNTY.KD.ZG,China,CHN,1970.0,NaN
Adjusted net national income (annual % growth),NY.ADJ.NNTY.KD.ZG,India,IND,1970.0,NaN
Adjusted net national income (annual % growth),NY.ADJ.NNTY.KD.ZG,Russian Federation,RUS,1970.0,NaN
Adjusted net national income (annual % growth),NY.ADJ.NNTY.KD.ZG,South Africa,ZAF,1970.0,NaN
Adjusted net national income (constant 2010 US$),NY.ADJ.NNTY.KD,Brazil,BRA,1970.0,3.918974e+11
Adjusted net national income (constant 2010 US$),NY.ADJ.NNTY.KD,China,CHN,1970.0,NaN
Adjusted net national income (constant 2010 US$),NY.ADJ.NNTY.KD,India,IND,1970.0,1.915335e+11
Adjusted net national income (constant 2010 US$),NY.ADJ.NNTY.KD,Russian Federation,RUS,1970.0,NaN


In [5]:
# Looking at the shape of the df_economy dataframe and datatypes of the columns/features
print(df_economy.shape)
df_economy.dtypes

(86500, 5)


SeriesCode      object
CountryName     object
CountryCode     object
Year           float64
Value          float64
dtype: object

From the initial peak into the dataset, I can see that some things need to be changed in order to perform machine learning and prediction models. First, I must drop the SeriesCode column as it provides no statistical information for the machine learning process. With only four features in the dataset after dropping, I plan on pivoting the table and making each unique SeriesName a feature for analysis with the rows as the Country and Year. In theory, this should make the dataset more applicable for machine learning models. Further, we can see that there are currently three features that are non-numeric, which will have to be addressed in the cleaning. Before, pivoting the table, I plan on understanding and preparing the data more by checking for null values and duplicated rows in each dataset. 

In [4]:
df_economy = df_economy.drop(['SeriesCode'], axis=1)
df_economy.head(10)

,CountryName,CountryCode,Year,Value
SeriesName,,,,
Adjusted net national income (annual % growth),Brazil,BRA,1970.0,NaN
Adjusted net national income (annual % growth),China,CHN,1970.0,NaN
Adjusted net national income (annual % growth),India,IND,1970.0,NaN
Adjusted net national income (annual % growth),Russian Federation,RUS,1970.0,NaN
Adjusted net national income (annual % growth),South Africa,ZAF,1970.0,NaN
Adjusted net national income (constant 2010 US$),Brazil,BRA,1970.0,3.918974e+11
Adjusted net national income (constant 2010 US$),China,CHN,1970.0,NaN
Adjusted net national income (constant 2010 US$),India,IND,1970.0,1.915335e+11
Adjusted net national income (constant 2010 US$),Russian Federation,RUS,1970.0,NaN


In [25]:
# Checking the number of null values within the Economy dataset
df_economy.isna().sum()

CountryName      250
CountryCode      250
Year               0
Value          26152
dtype: int64

In [26]:
# Returns the percentage of rows which have NaNs for each column.
df_economy.isna().sum()/len(df_economy)*100.0 

CountryName     0.289017
CountryCode     0.289017
Year            0.000000
Value          30.233526
dtype: float64

In [31]:
# Checking to see the number of duplicated rows in the Economy dataset
df_economy.duplicated().sum()

27793

In [32]:
# Returns the percentage of rows that were duplicated in the Economy dataset.
df_economy.duplicated().sum()/len(df_economy)*100.0

32.13063583815029

We can see that there is a high proportion of null values and duplicated rows in the dataset. Prior to downloading this data, I noticed that the Kaggle data originally pulled from The World Bank's DataBank. This database contains many databases that encompass different development indicators, growing industries, and statistical data related to population dynamics. Exploring these databases, I realized that for many of the indicators, the time series had a major impact on how many values were represented in the data. Considering this fact, I decided for my analysis of BRICS economies, I would focus on the time period starting in 1995 to 2018, as the database has not been updated for 2019 yet. I believe the reason The World Bank does not release current data and the datasets have many missing values is due to the inability to maintain a flow of survey data from some countries every year. For more info on why there are NaN values The World Bank explains the reason on this webpage, `https://datahelpdesk.worldbank.org/knowledgebase/articles/191133-why-are-some-data-not-available`. Before, removing the NaN values and duplicated columns, I will pivot the table in order to see how the number of rows and features change. 

In [8]:
df_economy_reshape = pd.pivot_table(df_economy, values = 'Value', index = ['CountryName','CountryCode','Year'], columns = 'SeriesName')
print(f'The size of the reshaped df_economy: {df_economy_reshape.shape}')
df_economy_reshape.head(10)

The size of the reshaped df_economy: (250, 332)


SeriesName                      Adjusted net national income (annual % growth)  \
CountryName CountryCode Year                                                     
Brazil      BRA         1970.0                                             NaN   
                        1971.0                                       11.642225   
                        1972.0                                       12.379004   
                        1973.0                                       12.214610   
                        1974.0                                        5.160031   
                        1975.0                                        2.928050   
                        1976.0                                       13.847055   
                        1977.0                                        6.897196   
                        1978.0                                        1.727041   
                        1979.0                                        6.489032   

SeriesName                      Adjusted net national income (constant 2010 US$)  \
CountryName CountryCode Year                                                       
Brazil      BRA         1970.0                                      3.918974e+11   
                        1971.0                                      4.375230e+11   
                        1972.0                                      4.916840e+11   
                        1973.0                                      5.517413e+11   
                        1974.0                                      5.802113e+11   
                        1975.0                                      5.972002e+11   
                        1976.0                                      6.798948e+11   
                        1977.0                                      7.267885e+11   
                        1978.0                                      7.393404e+11   
                        1979.0                                      7.873164e+11   

SeriesName                      Adjusted net national income (current US$)  \
CountryName CountryCode Year                                                 
Brazil      BRA         1970.0                                3.786021e+10   
                        1971.0                                4.414620e+10   
                        1972.0                                5.252340e+10   
                        1973.0                                7.058820e+10   
                        1974.0                                9.341705e+10   
                        1975.0                                1.086838e+11   
                        1976.0                                1.336018e+11   
                        1977.0                                1.544139e+11   
                        1978.0                                1.743500e+11   
                        1979.0                                1.936141e+11   

SeriesName                      Adjusted net national income per capita (annual % growth)  \
CountryName CountryCode Year                                                                
Brazil      BRA         1970.0                                                NaN           
                        1971.0                                           8.928380           
                        1972.0                                           9.704598           
                        1973.0                                           9.580841           
                        1974.0                                           2.702725           
                        1975.0                                           0.518431           
                        1976.0                                          11.177201           
                        1977.0                                           4.392284           
                        1978.0                                          -0.655074           
                        1979.0                

This reshaped dataframe enables us to do an analysis based on the many features present in the Economic Indicator data for each BRICS country over the years. From the shape of the dataset, I noticed that since we are analyzing 5 countries there are 50 years of data in the dataframe above from 1970 to 2020, 250/5 = 50. If we reduce the number of years, this will significantly reduce the number of rows in our dataset and merging the other four datasets will only add more features to the combined dataframe. Next, I will complete the steps above for the df_economy dataframe and apply it to the other 4 dataframes.

In [28]:
# Creating a CSV file for the cleaning section
df_economy_reshape.to_csv(r'C:\Users\Jim\Desktop\Economy_Rotated.csv')

In [21]:
# Taking a look into the Education_And_Environ_Data.csv
df_education.head(10)

,SeriesCode,CountryName,CountryCode,Year,Value
SeriesName,,,,,
Access to clean fuels and technologies for cooking (% of population),EG.CFT.ACCS.ZS,Brazil,BRA,1970.0,NaN
Access to clean fuels and technologies for cooking (% of population),EG.CFT.ACCS.ZS,China,CHN,1970.0,NaN
Access to clean fuels and technologies for cooking (% of population),EG.CFT.ACCS.ZS,India,IND,1970.0,NaN
Access to clean fuels and technologies for cooking (% of population),EG.CFT.ACCS.ZS,Russian Federation,RUS,1970.0,NaN
Access to clean fuels and technologies for cooking (% of population),EG.CFT.ACCS.ZS,South Africa,ZAF,1970.0,NaN
Access to electricity (% of population),EG.ELC.ACCS.ZS,Brazil,BRA,1970.0,NaN
Access to electricity (% of population),EG.ELC.ACCS.ZS,China,CHN,1970.0,NaN
Access to electricity (% of population),EG.ELC.ACCS.ZS,India,IND,1970.0,NaN
Access to electricity (% of population),EG.ELC.ACCS.ZS,Russian Federation,RUS,1970.0,NaN


In [13]:
# Looking at the shape of the df_education dataframe and datatypes of the columns/features
print(df_education.shape)
df_education.dtypes

(71500, 5)


SeriesCode      object
CountryName     object
CountryCode     object
Year           float64
Value          float64
dtype: object

We can see a similar pattern as the df_economy dataframe. The datatypes are the same and the shape of the dataset is smaller by approximately 15,000 values. Again, I will remove the SeriesCode column as I have decided that it will not be useful in my analysis and adds a non-numeric feature to the dataframes. After I remove the SeriesCode column, next I will check for null values and duplicated rows.

In [9]:
df_education = df_education.drop(['SeriesCode'], axis=1)
df_education.head(10)

,CountryName,CountryCode,Year,Value
SeriesName,,,,
Access to clean fuels and technologies for cooking (% of population),Brazil,BRA,1970.0,NaN
Access to clean fuels and technologies for cooking (% of population),China,CHN,1970.0,NaN
Access to clean fuels and technologies for cooking (% of population),India,IND,1970.0,NaN
Access to clean fuels and technologies for cooking (% of population),Russian Federation,RUS,1970.0,NaN
Access to clean fuels and technologies for cooking (% of population),South Africa,ZAF,1970.0,NaN
Access to electricity (% of population),Brazil,BRA,1970.0,NaN
Access to electricity (% of population),China,CHN,1970.0,NaN
Access to electricity (% of population),India,IND,1970.0,NaN
Access to electricity (% of population),Russian Federation,RUS,1970.0,NaN


In [41]:
# Checking the number of null values within the Education dataset
df_education.isna().sum()

CountryName      250
CountryCode      250
Year               0
Value          41640
dtype: int64

In [42]:
# Returns the percentage of rows which have NaNs for each column.
df_education.isna().sum()/len(df_education)*100.0 

CountryName     0.349650
CountryCode     0.349650
Year            0.000000
Value          58.237762
dtype: float64

In [43]:
# Checking to see the number of duplicated rows in the Economy dataset
df_education.duplicated().sum()

42085

In [44]:
# Returns the percentage of rows that were duplicated in the Economy dataset.
df_education.duplicated().sum()/len(df_education)*100.0

58.86013986013986

For the Education dataset, we can see that there are the same number of CountryName and CountryCode missing values as in the Economy dataset. Further, there is a higher proportion of missing values and duplicated rows in this dataset. In order to address the large proportion of data not present, I will have to select the specific timeline as discussed above in the cleaning section. For now, I will prepare the data by pivoting it in the same format as the first pivot `df_economy_reshape`.

In [10]:
df_education_reshape = pd.pivot_table(df_education, values = 'Value', index = ['CountryName','CountryCode','Year'], columns = 'SeriesName')
print(f'The size of the reshaped df_education: {df_education_reshape.shape}')
df_education_reshape.head(10)

The size of the reshaped df_education: (250, 285)


SeriesName                      Access to clean fuels and technologies for cooking (% of population)  \
CountryName CountryCode Year                                                                           
Brazil      BRA         1970.0                                                NaN                      
                        1971.0                                                NaN                      
                        1972.0                                                NaN                      
                        1973.0                                                NaN                      
                        1974.0                                                NaN                      
                        1975.0                                                NaN                      
                        1976.0                                                NaN                      
                        1977.0                                                NaN                      
                        1978.0                                                NaN                      
                        1979.0                                                NaN                      

SeriesName                      Access to electricity (% of population)  \
CountryName CountryCode Year                                              
Brazil      BRA         1970.0                                      NaN   
                        1971.0                                      NaN   
                        1972.0                                      NaN   
                        1973.0                                      NaN   
                        1974.0                                      NaN   
                        1975.0                                      NaN   
                        1976.0                                      NaN   
                        1977.0                                      NaN   
                        1978.0                                      NaN   
                        1979.0                                      NaN   

SeriesName                      Access to electricity, rural (% of rural population)  \
CountryName CountryCode Year                                                           
Brazil      BRA         1970.0                                                NaN      
                        1971.0                                                NaN      
                        1972.0                                                NaN      
                        1973.0                                                NaN      
                        1974.0                                                NaN      
                        1975.0                                                NaN      
                        1976.0                                                NaN      
                        1977.0                                                NaN      
                        1978.0                                                NaN      
                        1979.0                                                NaN      

SeriesName                      Access to electricity, urban (% of urban population)  \
CountryName CountryCode Year                                                           
Brazil      BRA         1970.0                                                NaN      
                        1971.0                                                NaN      
                        1972.0                                                NaN      
                        1973.0                                                NaN      
                        1974.0                                                NaN      
                        1975.0                                                NaN      
                        1976.0                                                NaN      
 

For this dataset, we can see that there are many NaN values for certain features and some features do not contain any. When cleaning the data and choosing the appropriate columns for the analysis, I will drop the columns that do not have values and keep the features that have enough data. I believe that many of these NaN values come from the fact that some of the data did not start pulling from survey until a certain year. I will have to find the year The World Bank started pulling the data. Now, I proceed to the Public dataset, df_public.

In [29]:
# Creating a CSV file for the cleaning section
df_education_reshape.to_csv(r'C:\Users\Jim\Desktop\Education_Rotated.csv')

In [22]:
# Taking a look into the Public_Sector_Indicators.csv
df_public.head(10)

,SeriesCode,CountryName,CountryCode,Year,Value
SeriesName,,,,,
Adequacy of social insurance programs (% of total welfare of beneficiary households),per_si_allsi.adq_pop_tot,Brazil,BRA,1970.0,NaN
Adequacy of social insurance programs (% of total welfare of beneficiary households),per_si_allsi.adq_pop_tot,China,CHN,1970.0,NaN
Adequacy of social insurance programs (% of total welfare of beneficiary households),per_si_allsi.adq_pop_tot,India,IND,1970.0,NaN
Adequacy of social insurance programs (% of total welfare of beneficiary households),per_si_allsi.adq_pop_tot,Russian Federation,RUS,1970.0,NaN
Adequacy of social insurance programs (% of total welfare of beneficiary households),per_si_allsi.adq_pop_tot,South Africa,ZAF,1970.0,NaN
Adequacy of social protection and labor programs (% of total welfare of beneficiary households),per_allsp.adq_pop_tot,Brazil,BRA,1970.0,NaN
Adequacy of social protection and labor programs (% of total welfare of beneficiary households),per_allsp.adq_pop_tot,China,CHN,1970.0,NaN
Adequacy of social protection and labor programs (% of total welfare of beneficiary households),per_allsp.adq_pop_tot,India,IND,1970.0,NaN
Adequacy of social protection and labor programs (% of total welfare of beneficiary households),per_allsp.adq_pop_tot,Russian Federation,RUS,1970.0,NaN


In [15]:
# Looking at the shape of the df_public dataframe and datatypes of the columns/features
print(df_public.shape)
df_public.dtypes

(81500, 5)


SeriesCode      object
CountryName     object
CountryCode     object
Year           float64
Value          float64
dtype: object

Again, I will remove the SeriesCode column and check for null values and duplicated rows below.

In [12]:
df_public = df_public.drop(['SeriesCode'], axis=1)
df_public.head(10)

,CountryName,CountryCode,Year,Value
SeriesName,,,,
Adequacy of social insurance programs (% of total welfare of beneficiary households),Brazil,BRA,1970.0,NaN
Adequacy of social insurance programs (% of total welfare of beneficiary households),China,CHN,1970.0,NaN
Adequacy of social insurance programs (% of total welfare of beneficiary households),India,IND,1970.0,NaN
Adequacy of social insurance programs (% of total welfare of beneficiary households),Russian Federation,RUS,1970.0,NaN
Adequacy of social insurance programs (% of total welfare of beneficiary households),South Africa,ZAF,1970.0,NaN
Adequacy of social protection and labor programs (% of total welfare of beneficiary households),Brazil,BRA,1970.0,NaN
Adequacy of social protection and labor programs (% of total welfare of beneficiary households),China,CHN,1970.0,NaN
Adequacy of social protection and labor programs (% of total welfare of beneficiary households),India,IND,1970.0,NaN
Adequacy of social protection and labor programs (% of total welfare of beneficiary households),Russian Federation,RUS,1970.0,NaN


In [47]:
# Checking the number of null values within the Public dataset
df_public.isna().sum()

CountryName      250
CountryCode      250
Year               0
Value          54662
dtype: int64

In [48]:
# Returns the percentage of rows which have NaNs for each column.
df_public.isna().sum()/len(df_public)*100.0 

CountryName     0.306748
CountryCode     0.306748
Year            0.000000
Value          67.069939
dtype: float64

In [49]:
# Checking to see the number of duplicated rows in the Economy dataset
df_public.duplicated().sum()

54816

In [50]:
# Returns the percentage of rows that were duplicated in the Economy dataset.
df_public.duplicated().sum()/len(df_public)*100.0

67.25889570552147

The same theory applies to the Public dataset as the data was most likely formatted to contain the 50 year time series, when some of the data was not pulled during this time period. I will need to account more for the time as the proportion of NaN values and duplicated rows is higher here. An additional note, I believe that some of these duplicated rows are the product of the rows with NaN values. This will need to be proven though in the cleaning section. Now, I will pivot the table in the mentioned format.

In [13]:
df_public_reshape = pd.pivot_table(df_public, values = 'Value', index = ['CountryName','CountryCode','Year'], columns = 'SeriesName')
print(f'The size of the reshaped df_public: {df_public_reshape.shape}')
df_public_reshape.head(10)

The size of the reshaped df_public: (250, 316)


SeriesName                      Adequacy of social insurance programs (% of total welfare of beneficiary households)  \
CountryName CountryCode Year                                                                                           
Brazil      BRA         1970.0                                                NaN                                      
                        1971.0                                                NaN                                      
                        1972.0                                                NaN                                      
                        1973.0                                                NaN                                      
                        1974.0                                                NaN                                      
                        1975.0                                                NaN                                      
                        1976.0                                                NaN                                      
                        1977.0                                                NaN                                      
                        1978.0                                                NaN                                      
                        1979.0                                                NaN                                      

SeriesName                      Adequacy of social protection and labor programs (% of total welfare of beneficiary households)  \
CountryName CountryCode Year                                                                                                      
Brazil      BRA         1970.0                                                NaN                                                 
                        1971.0                                                NaN                                                 
                        1972.0                                                NaN                                                 
                        1973.0                                                NaN                                                 
                        1974.0                                                NaN                                                 
                        1975.0                                                NaN                                                 
                        1976.0                                                NaN                                                 
                        1977.0                                                NaN                                                 
                        1978.0                                                NaN                                                 
                        1979.0                                                NaN                                                 

SeriesName                      Adequacy of social safety net programs (% of total welfare of beneficiary households)  \
CountryName CountryCode Year                                                                                            
Brazil      BRA         1970.0                                                NaN                                       
                        1971.0                                                NaN                                       
                        1972.0                                                NaN                                       
                        1973.0                                                NaN                                       
                        1974.0                                                NaN                                       
                        1975.0                                                NaN                                       
                  

We can see a similar pattern as the df_education_reshape dataframe as some features collect data since 1970, where many features do not. Next, I will prepare the Private dataset, df_private.

In [30]:
# Creating a CSV file for the cleaning section
df_public_reshape.to_csv(r'C:\Users\Jim\Desktop\Public_Rotated.csv')

In [23]:
# Taking a look into the Private_Sector_Data.csv
df_private.head(10)

,SeriesCode,CountryName,CountryCode,Year,Value
SeriesName,,,,,
Agricultural raw materials exports (% of merchandise exports),TX.VAL.AGRI.ZS.UN,Brazil,BRA,1970.0,11.895098
Agricultural raw materials exports (% of merchandise exports),TX.VAL.AGRI.ZS.UN,China,CHN,1970.0,NaN
Agricultural raw materials exports (% of merchandise exports),TX.VAL.AGRI.ZS.UN,India,IND,1970.0,5.556494
Agricultural raw materials exports (% of merchandise exports),TX.VAL.AGRI.ZS.UN,Russian Federation,RUS,1970.0,NaN
Agricultural raw materials exports (% of merchandise exports),TX.VAL.AGRI.ZS.UN,South Africa,ZAF,1970.0,NaN
Agricultural raw materials imports (% of merchandise imports),TM.VAL.AGRI.ZS.UN,Brazil,BRA,1970.0,1.851044
Agricultural raw materials imports (% of merchandise imports),TM.VAL.AGRI.ZS.UN,China,CHN,1970.0,NaN
Agricultural raw materials imports (% of merchandise imports),TM.VAL.AGRI.ZS.UN,India,IND,1970.0,9.207277
Agricultural raw materials imports (% of merchandise imports),TM.VAL.AGRI.ZS.UN,Russian Federation,RUS,1970.0,NaN


In [19]:
# Looking at the shape of the df_private dataframe and datatypes of the columns/features
print(df_private.shape)
df_private.dtypes

(42250, 5)


SeriesCode      object
CountryName     object
CountryCode     object
Year           float64
Value          float64
dtype: object

First noticable attribute of this dataframe is it contains less values than the other dataframes. This tells me that the Private dataset will have less features when pivoting. I will remove the SeriesCode next.

In [14]:
df_private = df_private.drop(['SeriesCode'], axis=1)
df_private.head(10)

,CountryName,CountryCode,Year,Value
SeriesName,,,,
Agricultural raw materials exports (% of merchandise exports),Brazil,BRA,1970.0,11.895098
Agricultural raw materials exports (% of merchandise exports),China,CHN,1970.0,NaN
Agricultural raw materials exports (% of merchandise exports),India,IND,1970.0,5.556494
Agricultural raw materials exports (% of merchandise exports),Russian Federation,RUS,1970.0,NaN
Agricultural raw materials exports (% of merchandise exports),South Africa,ZAF,1970.0,NaN
Agricultural raw materials imports (% of merchandise imports),Brazil,BRA,1970.0,1.851044
Agricultural raw materials imports (% of merchandise imports),China,CHN,1970.0,NaN
Agricultural raw materials imports (% of merchandise imports),India,IND,1970.0,9.207277
Agricultural raw materials imports (% of merchandise imports),Russian Federation,RUS,1970.0,NaN


In [53]:
# Checking the number of null values within the Private dataset
df_private.isna().sum()

CountryName      250
CountryCode      250
Year               0
Value          24830
dtype: int64

In [54]:
# Returns the percentage of rows which have NaNs for each column.
df_private.isna().sum()/len(df_private)*100.0 

CountryName     0.591716
CountryCode     0.591716
Year            0.000000
Value          58.769231
dtype: float64

In [55]:
# Checking to see the number of duplicated rows in the Economy dataset
df_private.duplicated().sum()

25907

In [56]:
# Returns the percentage of rows that were duplicated in the Economy dataset.
df_private.duplicated().sum()/len(df_private)*100.0

61.318343195266266

Again, we see a high proportion of Nan values and duplicated rows within the Private dataset. I will now pivot the dataframe in the specified format.

In [15]:
df_private_reshape = pd.pivot_table(df_private, values = 'Value', index = ['CountryName','CountryCode','Year'], columns = 'SeriesName')
print(f'The size of the reshaped df_private: {df_private_reshape.shape}')
df_private_reshape.head(10)

The size of the reshaped df_private: (230, 168)


SeriesName                      Agricultural raw materials exports (% of merchandise exports)  \
CountryName CountryCode Year                                                                    
Brazil      BRA         1970.0                                          11.895098               
                        1971.0                                          10.494023               
                        1972.0                                           9.322425               
                        1973.0                                           8.521143               
                        1974.0                                           5.991149               
                        1975.0                                           3.891622               
                        1976.0                                           2.273232               
                        1977.0                                           2.353515               
                        1978.0                                           2.918907               
                        1979.0                                           3.442275               

SeriesName                      Agricultural raw materials imports (% of merchandise imports)  \
CountryName CountryCode Year                                                                    
Brazil      BRA         1970.0                                           1.851044               
                        1971.0                                           2.178320               
                        1972.0                                           1.971959               
                        1973.0                                           2.073260               
                        1974.0                                           2.014877               
                        1975.0                                           1.497433               
                        1976.0                                           1.418146               
                        1977.0                                           1.458700               
                        1978.0                                           1.447775               
                        1979.0                                           1.351869               

SeriesName                      Average number of visits or required meetings with tax officials (for affected firms)  \
CountryName CountryCode Year                                                                                            
Brazil      BRA         1970.0                                                NaN                                       
                        1971.0                                                NaN                                       
                        1972.0                                                NaN                                       
                        1973.0                                                NaN                                       
                        1974.0                                                NaN                                       
                        1975.0                                                NaN                                       
                        1976.0                                                NaN                                       
                        1977.0                                                NaN                                       
                        1978.0                                                NaN                                       
                        1979.0                                                NaN                                       

SeriesName                      Average time to clear exports through customs (days)  \
CountryName CountryCode Year                                                           
Brazil      BRA         1970.0           

As guessed, the Private dataset contains less features than the other datasets as the number of values was significantly lower. An important note is the fact that there are only 230 rows compared to the other 3 dataframes. This suggests that there are 20 rows of missing data for a specific time series somewhere in the dataframe. I will need to find these values and understand why they are excluded from the dataset. When combining all the dataframes into one, I will need to understand which features are relevant and contain information for the time period I plan to analyze. Finally, I will prepare and gain an understanding of the Health dataset.

In [31]:
# Creating a CSV file for the cleaning section
df_private_reshape.to_csv(r'C:\Users\Jim\Desktop\Private_Rotated.csv')

In [24]:
# Taking a look into the Health_And_Poverty_Data.csv
df_health.head(10)

,SeriesCode,CountryName,CountryCode,Year,Value
SeriesName,,,,,
"Adolescent fertility rate (births per 1,000 women ages 15-19)",SP.ADO.TFRT,Brazil,BRA,1970.0,77.1184
"Adolescent fertility rate (births per 1,000 women ages 15-19)",SP.ADO.TFRT,China,CHN,1970.0,38.6866
"Adolescent fertility rate (births per 1,000 women ages 15-19)",SP.ADO.TFRT,India,IND,1970.0,108.3178
"Adolescent fertility rate (births per 1,000 women ages 15-19)",SP.ADO.TFRT,Russian Federation,RUS,1970.0,29.8818
"Adolescent fertility rate (births per 1,000 women ages 15-19)",SP.ADO.TFRT,South Africa,ZAF,1970.0,93.7106
Adults (ages 15+) and children (ages 0-14) newly infected with HIV,SH.HIV.INCD.TL,Brazil,BRA,1970.0,NaN
Adults (ages 15+) and children (ages 0-14) newly infected with HIV,SH.HIV.INCD.TL,China,CHN,1970.0,NaN
Adults (ages 15+) and children (ages 0-14) newly infected with HIV,SH.HIV.INCD.TL,India,IND,1970.0,NaN
Adults (ages 15+) and children (ages 0-14) newly infected with HIV,SH.HIV.INCD.TL,Russian Federation,RUS,1970.0,NaN


In [18]:
# Looking at the shape of the df_health dataframe and datatypes of the columns/features
print(df_health.shape)
df_health.dtypes

(69500, 5)


SeriesCode      object
CountryName     object
CountryCode     object
Year           float64
Value          float64
dtype: object

The size of the Health dataframe tells me that it is a relatively sizable dataset and I will now remove the SeriesCode column as I have for the other 4 datasets.

In [17]:
df_health = df_health.drop(['SeriesCode'], axis=1)
df_health.head(10)

,CountryName,CountryCode,Year,Value
SeriesName,,,,
"Adolescent fertility rate (births per 1,000 women ages 15-19)",Brazil,BRA,1970.0,77.1184
"Adolescent fertility rate (births per 1,000 women ages 15-19)",China,CHN,1970.0,38.6866
"Adolescent fertility rate (births per 1,000 women ages 15-19)",India,IND,1970.0,108.3178
"Adolescent fertility rate (births per 1,000 women ages 15-19)",Russian Federation,RUS,1970.0,29.8818
"Adolescent fertility rate (births per 1,000 women ages 15-19)",South Africa,ZAF,1970.0,93.7106
Adults (ages 15+) and children (ages 0-14) newly infected with HIV,Brazil,BRA,1970.0,NaN
Adults (ages 15+) and children (ages 0-14) newly infected with HIV,China,CHN,1970.0,NaN
Adults (ages 15+) and children (ages 0-14) newly infected with HIV,India,IND,1970.0,NaN
Adults (ages 15+) and children (ages 0-14) newly infected with HIV,Russian Federation,RUS,1970.0,NaN


In [62]:
# Checking the number of null values within the Health dataset
df_health.isna().sum()

CountryName      250
CountryCode      250
Year               0
Value          39481
dtype: int64

In [63]:
# Returns the percentage of rows which have NaNs for each column.
df_health.isna().sum()/len(df_health)*100.0 

CountryName     0.359712
CountryCode     0.359712
Year            0.000000
Value          56.807194
dtype: float64

In [64]:
# Checking to see the number of duplicated rows in the Economy dataset
df_health.duplicated().sum()

39632

In [65]:
# Returns the percentage of rows that were duplicated in the Economy dataset.
df_health.duplicated().sum()/len(df_health)*100.0

57.024460431654674

In [18]:
df_health_reshape = pd.pivot_table(df_health, values = 'Value', index = ['CountryName','CountryCode','Year'], columns = 'SeriesName')
print(f'The size of the reshaped df_health: {df_health_reshape.shape}')
df_health_reshape.head(10)

The size of the reshaped df_health: (250, 263)


SeriesName                      ARI treatment (% of children under 5 taken to a health provider)  \
CountryName CountryCode Year                                                                       
Brazil      BRA         1970.0                                                NaN                  
                        1971.0                                                NaN                  
                        1972.0                                                NaN                  
                        1973.0                                                NaN                  
                        1974.0                                                NaN                  
                        1975.0                                                NaN                  
                        1976.0                                                NaN                  
                        1977.0                                                NaN                  
                        1978.0                                                NaN                  
                        1979.0                                                NaN                  

SeriesName                      Adolescent fertility rate (births per 1,000 women ages 15-19)  \
CountryName CountryCode Year                                                                    
Brazil      BRA         1970.0                                            77.1184               
                        1971.0                                            75.4532               
                        1972.0                                            73.7880               
                        1973.0                                            73.8080               
                        1974.0                                            73.8280               
                        1975.0                                            73.8480               
                        1976.0                                            73.8680               
                        1977.0                                            73.8880               
                        1978.0                                            75.0802               
                        1979.0                                            76.2724               

SeriesName                      Adults (ages 15+) and children (ages 0-14) newly infected with HIV  \
CountryName CountryCode Year                                                                         
Brazil      BRA         1970.0                                                NaN                    
                        1971.0                                                NaN                    
                        1972.0                                                NaN                    
                        1973.0                                                NaN                    
                        1974.0                                                NaN                    
                        1975.0                                                NaN                    
                        1976.0                                                NaN                    
                        1977.0                                                NaN                    
                        1978.0                                                NaN                    
                        1979.0                                                NaN                    

SeriesName                      Adults (ages 15-49) newly infected with HIV  \
CountryName CountryCode Year                                                  
Brazil      BRA         1970.0                                          NaN   
                        1971.0                                          NaN   
                        1972.0                                          NaN   
              

Looking at the new Health dataframe, we can see that there are the same number of rows as with the other 3 datasets, indicating the same 50 year period. This will need to be adjusted according to the time series before cleaning, in order to allocate the correct cleaning methods and framework. With the final dataframe pivoted in the same way as the other 4 dataframes, I can now clean the dataset and allocate my time series in order to make a combined dataframe that can be used for the machine learning analysis. I will now clean each dataframe and explain my reasoning as to why I performed the actions I did.

In [32]:
# Creating a CSV file for the cleaning section
df_health_reshape.to_csv(r'C:\Users\Jim\Desktop\Health_Rotated.csv')

## Cleaning (Next Notebook)
With the data prepared for cleaning, now I can proceed to clean all the dataframes according to my time series and combine the dataframes into an aggregated dataframe.

In [67]:
df_economy_reshape

SeriesName                       Adjusted net national income (annual % growth)  \
CountryName  CountryCode Year                                                     
Brazil       BRA         1970.0                                             NaN   
                         1971.0                                       11.642225   
                         1972.0                                       12.379004   
                         1973.0                                       12.214610   
                         1974.0                                        5.160031   
...                                                                         ...   
South Africa ZAF         2015.0                                        3.600109   
                         2016.0                                       -0.103042   
                         2017.0                                        2.581799   
                         2018.0                                       -0.093618   
                         2019.0                                             NaN   

SeriesName                       Adjusted net national income (constant 2010 US$)  \
CountryName  CountryCode Year                                                       
Brazil       BRA         1970.0                                      3.918974e+11   
                         1971.0                                      4.375230e+11   
                         1972.0                                      4.916840e+11   
                         1973.0                                      5.517413e+11   
                         1974.0                                      5.802113e+11   
...                                                                           ...   
South Africa ZAF         2015.0                                      3.438474e+11   
                         2016.0                                      3.434931e+11   
                         2017.0                                      3.523614e+11   
                         2018.0                                      3.520315e+11   
                         2019.0                                               NaN   

SeriesName                       Adjusted net national income (current US$)  \
CountryName  CountryCode Year                                                 
Brazil       BRA         1970.0                                3.786021e+10   
                         1971.0                                4.414620e+10   
                         1972.0                                5.252340e+10   
                         1973.0                                7.058820e+10   
                         1974.0                                9.341705e+10   
...                                                                     ...   
South Africa ZAF         2015.0                                2.588321e+11   
                         2016.0                                2.389447e+11   
                         2017.0                                2.825657e+11   
                         2018.0                                2.956752e+11   
                         2019.0                                         NaN   

SeriesName                       Adjusted net national income per capita (annual % growth)  \
CountryName  CountryCode Year                                                                
Brazil       BRA         1970.0                                                NaN           
                         1971.0                                           8.928380           
                         1972.0                                           9.704598           
                         1973.0                                           9.580841           
                         1974.0                                           2.702725           
...                                                                            ...           
South Africa ZAF         2015.0   

In [68]:
df_education_reshape

SeriesName                       Access to clean fuels and technologies for cooking (% of population)  \
CountryName  CountryCode Year                                                                           
Brazil       BRA         1970.0                                                NaN                      
                         1971.0                                                NaN                      
                         1972.0                                                NaN                      
                         1973.0                                                NaN                      
                         1974.0                                                NaN                      
...                                                                            ...                      
South Africa ZAF         2015.0                                              83.64                      
                         2016.0                                              84.75                      
                         2017.0                                                NaN                      
                         2018.0                                                NaN                      
                         2019.0                                                NaN                      

SeriesName                       Access to electricity (% of population)  \
CountryName  CountryCode Year                                              
Brazil       BRA         1970.0                                      NaN   
                         1971.0                                      NaN   
                         1972.0                                      NaN   
                         1973.0                                      NaN   
                         1974.0                                      NaN   
...                                                                  ...   
South Africa ZAF         2015.0                                85.500000   
                         2016.0                                84.200000   
                         2017.0                                84.400000   
                         2018.0                                91.229874   
                         2019.0                                      NaN   

SeriesName                       Access to electricity, rural (% of rural population)  \
CountryName  CountryCode Year                                                           
Brazil       BRA         1970.0                                                NaN      
                         1971.0                                                NaN      
                         1972.0                                                NaN      
                         1973.0                                                NaN      
                         1974.0                                                NaN      
...                                                                            ...      
South Africa ZAF         2015.0                                          75.363531      
                         2016.0                                          70.715722      
                         2017.0                                          70.314530      
                         2018.0                                          89.598819      
                         2019.0                                                NaN      

SeriesName                       Access to electricity, urban (% of urban population)  \
CountryName  CountryCode Year                                                           
Brazil       BRA         1970.0                                                NaN      
                         1971.0                                                NaN      
                         1972.0                                                NaN      
                         1973.0           

In [69]:
df_public_reshape

SeriesName                       Adequacy of social insurance programs (% of total welfare of beneficiary households)  \
CountryName  CountryCode Year                                                                                           
Brazil       BRA         1970.0                                                NaN                                      
                         1971.0                                                NaN                                      
                         1972.0                                                NaN                                      
                         1973.0                                                NaN                                      
                         1974.0                                                NaN                                      
...                                                                            ...                                      
South Africa ZAF         2015.0                                                NaN                                      
                         2016.0                                                NaN                                      
                         2017.0                                                NaN                                      
                         2018.0                                                NaN                                      
                         2019.0                                                NaN                                      

SeriesName                       Adequacy of social protection and labor programs (% of total welfare of beneficiary households)  \
CountryName  CountryCode Year                                                                                                      
Brazil       BRA         1970.0                                                NaN                                                 
                         1971.0                                                NaN                                                 
                         1972.0                                                NaN                                                 
                         1973.0                                                NaN                                                 
                         1974.0                                                NaN                                                 
...                                                                            ...                                                 
South Africa ZAF         2015.0                                                NaN                                                 
                         2016.0                                                NaN                                                 
                         2017.0                                                NaN                                                 
                         2018.0                                                NaN                                                 
                         2019.0                                                NaN                                                 

SeriesName                       Adequacy of social safety net programs (% of total welfare of beneficiary households)  \
CountryName  CountryCode Year                                                                                            
Brazil       BRA         1970.0                                                NaN                                       
                         1971.0                                                NaN                                       
                         1972.0                                                NaN                                       
                         1973.0                                                NaN                 

In [70]:
df_private_reshape

SeriesName                       Agricultural raw materials exports (% of merchandise exports)  \
CountryName  CountryCode Year                                                                    
Brazil       BRA         1970.0                                          11.895098               
                         1971.0                                          10.494023               
                         1972.0                                           9.322425               
                         1973.0                                           8.521143               
                         1974.0                                           5.991149               
...                                                                            ...               
South Africa ZAF         2015.0                                           2.179561               
                         2016.0                                           2.351764               
                         2017.0                                           2.346479               
                         2018.0                                           2.322842               
                         2019.0                                           1.849867               

SeriesName                       Agricultural raw materials imports (% of merchandise imports)  \
CountryName  CountryCode Year                                                                    
Brazil       BRA         1970.0                                           1.851044               
                         1971.0                                           2.178320               
                         1972.0                                           1.971959               
                         1973.0                                           2.073260               
                         1974.0                                           2.014877               
...                                                                            ...               
South Africa ZAF         2015.0                                           0.920385               
                         2016.0                                           0.989361               
                         2017.0                                           0.991302               
                         2018.0                                           1.001650               
                         2019.0                                           0.965994               

SeriesName                       Average number of visits or required meetings with tax officials (for affected firms)  \
CountryName  CountryCode Year                                                                                            
Brazil       BRA         1970.0                                                NaN                                       
                         1971.0                                                NaN                                       
                         1972.0                                                NaN                                       
                         1973.0                                                NaN                                       
                         1974.0                                                NaN                                       
...                                                                            ...                                       
South Africa ZAF         2015.0                                                NaN                                       
                         2016.0                                                NaN                                       
                         2017.0                                                NaN                                       
                         2018.0                                                NaN                          

In [71]:
df_health_reshape

SeriesName                       ARI treatment (% of children under 5 taken to a health provider)  \
CountryName  CountryCode Year                                                                       
Brazil       BRA         1970.0                                                NaN                  
                         1971.0                                                NaN                  
                         1972.0                                                NaN                  
                         1973.0                                                NaN                  
                         1974.0                                                NaN                  
...                                                                            ...                  
South Africa ZAF         2015.0                                                NaN                  
                         2016.0                                               87.6                  
                         2017.0                                                NaN                  
                         2018.0                                                NaN                  
                         2019.0                                                NaN                  

SeriesName                       Adolescent fertility rate (births per 1,000 women ages 15-19)  \
CountryName  CountryCode Year                                                                    
Brazil       BRA         1970.0                                            77.1184               
                         1971.0                                            75.4532               
                         1972.0                                            73.7880               
                         1973.0                                            73.8080               
                         1974.0                                            73.8280               
...                                                                            ...               
South Africa ZAF         2015.0                                            69.5352               
                         2016.0                                            68.7216               
                         2017.0                                            67.9080               
                         2018.0                                            67.8488               
                         2019.0                                                NaN               

SeriesName                       Adults (ages 15+) and children (ages 0-14) newly infected with HIV  \
CountryName  CountryCode Year                                                                         
Brazil       BRA         1970.0                                                NaN                    
                         1971.0                                                NaN                    
                         1972.0                                                NaN                    
                         1973.0                                                NaN                    
                         1974.0                                                NaN                    
...                                                                            ...                    
South Africa ZAF         2015.0                                           280000.0                    
                         2016.0                                           260000.0                    
                         2017.0                                           240000.0                    
                         2018.0                                           220000.0                    
                         2019.0                                           200000.0                    

SeriesName                       Adults (ages 15-49) newly infected wit

Please move on to the next notebook `Capstone Project: BRICS Analysis Cleaning` for the cleaning process of these datasets and the combination of them.